# Random Forest for An Alignment Cost-Based Classification of Log Traces

# 0. Fitness

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd 
import numpy as np
np.random.seed(0)
import time

from statistics import mean

from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from tensorflow.keras.losses import BinaryCrossentropy
#tensorflow as tf
#from tensorflow import enable_eager_execution
#enable_eager_execution() 

In [2]:
def fitness(packageForFitness, minRunLength):
    '''
    This function computes the fitness for all the sequences. 
    @sequences: the sequences of words
    @costs: the real alignment cost
    @minRunLength: the minimal run in the alignment dataset
    '''
    sumTraceFitness = 0
    totTraces = 0 
    for i in packageForFitness.index:
        sumTraceFitness += (1 - (packageForFitness.realCosts[i] / ( packageForFitness.lengths[i]  + minRunLength )))#*packageForFitness.freqs[i]
        #totTraces += packageForFitness.freqs[i]
        totTraces +=1
    return sumTraceFitness / totTraces

def LB_fitness(packageForFitness, minRunLength, m_AC, indices):
    '''
    This function computes lower bound of the fitness given in the paper. 
    @sequences: the sequences of words
    @minRunLength: the minimal run in the alignment dataset
    @m_AC: needed for the lowerbound formula
    @indices: if we compute the lower bound, then we don't iterate on all the traces but only the positives. 
    '''
    sumTraceFitness = 0
    totTraces = 0 
    for i in indices:
        sumTraceFitness += (1 - ((m_AC) / ( packageForFitness.lengths[i] + minRunLength )))#*packageForFitness.freqs[i]
    for i in packageForFitness.index:
        #totTraces += packageForFitness.freqs[i]
        totTraces +=1
    return sumTraceFitness / totTraces

# 1. Preprocessing the data

In [3]:
# Initialize the "CountVectorizer" object as a GLOBAL VARIABLE, which is scikit-learn's bag of words tool.
vectorizer = CountVectorizer(analyzer = 'word',
                            tokenizer = None,
                            preprocessor = None,
                            stop_words = None,
                            max_features = 5000)

In [4]:
def cleanDataForRF(dataFile,m_AC,vectorizer_already_trained=None): 
    '''
    Reads the file (1), specifies the target classes (2) and prepare the Bag of Words(3).
    @dataFile: (String) filename of the alignment dataset
    @m_AC: (int) maximal alignment cost classifier
    '''
    # ---- (1) yes, a bit of copy/paste... Read the file 
    data = pd.read_csv(dataFile,sep = ";", 
                   names = ["traces","tracesWithMoves","runs","runsWithMoves","costs","frequencies"])
    
    # ---- (2) create the positive and negative target depending on the m_AC parameter
    # alignment cost which interests us is greater than 10000 (other costs are just silent moves)
    # set the fitting to tmp_pos to set them latter to 1
    y = ((data["costs"] / 10000) / (m_AC+1)).astype(int)
    max_y = y.max()
    y = y.replace(0,"tmp_pos")
    y = y.replace(range(1,max_y + 1), 0)
    y = y.replace("tmp_pos",1)
    y = np.eye(2)[y.to_numpy().reshape(-1)]
    
    # ---- (3) prepare the Bag of Words
    traces_to_matrix = data.traces.str.split(":::",expand=True,)
    # this line takes the matrix of words, and transformed it to a list of sentences
    data_to_fit = [' '.join( [e.replace(" ","") for e in filter(None,a)]) for a in traces_to_matrix.values.tolist()]
    
    if vectorizer_already_trained :
        x = vectorizer.transform(data_to_fit).toarray()
    else :
        # then we can transform our data with the counter (it's like a one-hot-encode, right?)
        x = vectorizer.fit_transform(data_to_fit).toarray()
    
    # for fitness computation
    minLengthRun= len(data.runs.str.split(":::").min())
    lengths = data.traces.str.split(":::",expand=False,).str.len()-1
    realCosts = (data["costs"] / 10000).astype(int)
    packageForFitness = pd.DataFrame({"lengths":lengths, "realCosts": realCosts, "freqs": data.frequencies})
    
    return x, y, packageForFitness, minLengthRun, m_AC

In [5]:
# example of use
x, y, packageForFitness, minLengthRun, m_AC = cleanDataForRF("alignments-A_2012_im.csv",4)
x, y

(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 5, 5, 0],
        [1, 0, 0, ..., 5, 3, 0],
        [1, 0, 0, ..., 3, 0, 0]], dtype=int64),
 array([[0., 1.],
        [0., 1.],
        [0., 1.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]]))

# 3. Cross-Validation of the method

In [6]:
def loss(forest, x_test, y_test):
    y_test_predict_with_proba = forest.predict_proba(x_test)[1]
    return  BinaryCrossentropy()(y_test,y_test_predict_with_proba).numpy()

def acc(forest, x_test, y_test):
    y_test_predict = forest.predict(x_test)
    return accuracy_score(y_test,y_test_predict) 

In [7]:
def accLossPercentage(forest, x, y, indices_of_test, accArr, lossArr, percentageArr=None, freqs=None,classToTest=None):
    '''
    This function fills the arrays of results accArr, lossArr and percentageArr for all the test items, or only the 
    negative items (classToTest=0) or only the positive items (classToTest=1). percentageArr is optional because it 
    is not required for the entire dataset, i.e., when we do not specify the class. 
    This works by using a dynamic programmation for the arrays. The return element is either the current accuracy in 
    case of classToTest=None, either the indices in case of classToTest!=None.
    Params:
    @forest: a trained model
    @x_test: the dataset to test
    @y_test: the target value to predict for the test dataset
    @accArr: a list of the previous accurary, or an empty list
    @lossArr: a list of the previous loss, or an empty list
    @percentageArr:  a list of the previous percentage, or an empty list
    @classToTest: 1 or 0 for positive and negative. This will find the indices of the items that belongs to the class
    '''
    if classToTest!=None:
        indices = [i for i in indices_of_test if y[i][1]==classToTest]
        if len(indices)>0:
            accArr.append(acc(forest, x[indices], y[indices]))
            lossArr.append(loss(forest, x[indices], y[indices]))
        #percentageArr.append(freqs[indices].sum()/freqs[indices_of_test].sum())
        percentageArr.append(len(indices)/len(indices_of_test))
        return indices
    else :
        accuracy = acc(forest, x[indices_of_test], y[indices_of_test])
        accArr.append(accuracy)
        lossArr.append(loss(forest, x[indices_of_test], y[indices_of_test]))
        return accuracy

In [8]:
def runKFoldForRF(numberOfFold,x,y,packageForFitness,minLengthRun, m_AC):
    accAll, lossAll = [], []
    accNeg, lossNeg, percentageNeg = [], [], []
    accPos, lossPos, percentagePos = [], [], []
    
    realFitness, realLBFitness, predictedLBFitness = [], [], []
    packageForFitness.reset_index(drop=True,inplace=True)
    
    runtime = []

    # use a K-fold Cross-Validation and show average Loss and average Accuracy
    kfold = KFold(numberOfFold,)
    bestmodel, bestAccuracy = None, 0
    
    for indices_of_train, indices_of_test in kfold.split(x):
        forest = RandomForestClassifier(n_estimators = 100) 
        forest.fit(x[indices_of_train], y[indices_of_train])
        
        # compute loss and accuracy on the test items
        current_accuracy = accLossPercentage(forest, x, y, indices_of_test, accAll, lossAll, )
        
        # compute loss and accuracy on the test items that are negatives
        accLossPercentage(forest, x, y, indices_of_test, accNeg, lossNeg, percentageNeg,packageForFitness.freqs, 0)

        # compute loss and accuracy on the test items that are positives
        indices_of_pos = accLossPercentage(forest, x, y, indices_of_test, accPos, lossPos, percentagePos,packageForFitness.freqs, 1) 

        # compute fitness and lower-bound
        realFitness.append(fitness(packageForFitness.iloc[indices_of_test], minLengthRun))
        
        # compute real LB, which depends on number of pos items
        if len(indices_of_pos)>0:
            realLBFitness.append(LB_fitness(packageForFitness.iloc[indices_of_test], minLengthRun, m_AC, indices_of_pos))
        
        # compute predicted LB fitness and runtime
        start = time.time()
        predictions = forest.predict(x[indices_of_test])
        runtime.append((time.time()-start)/len(indices_of_test))
        
        indices_of_predicted_as_positives = [indices_of_test[i] for i in range(0,len(predictions)) if predictions[i][1]==1]
        if len(indices_of_predicted_as_positives)>0:
            predictedLBFitness.append(LB_fitness(packageForFitness.iloc[indices_of_test], minLengthRun, m_AC, indices_of_predicted_as_positives))
            
        if bestAccuracy < current_accuracy:
            bestmodel = forest
            bestAccuracy = current_accuracy
            
    print("[CROSS-VALIDATION]\n[ALL] Loss:", "{:.3f}".format(mean(lossAll)), "\t Acc:", "{:.3f}".format(mean(accAll)))
    print("[POSITIVE ({:.2f}%)] Loss:".format(mean(percentagePos)), "{:.3f}".format(mean(lossPos)), "\t Acc:", "{:.3f}".format(mean(accPos)))
    print("[NEGATIVE ({:.2f}%)] Loss:".format(mean(percentageNeg)), "{:.3f}".format(mean(lossNeg)),"\t Acc:", "{:.3f}".format(mean(accNeg)))
    print("Fitness {:.3f}".format(mean(realFitness)), "\t LB Fitness:", "{:.3f}\n".format(mean(realLBFitness)),"\t LB Fitness:", "{:.3f}\n".format(mean(predictedLBFitness)))
    print("Runtime (prediction per trace):{:.10f}".format(mean(runtime)))
    return bestmodel

In [9]:
# example of use
runKFoldForRF(3,x,y,packageForFitness, minLengthRun, m_AC)

[CROSS-VALIDATION]
[ALL] Loss: 0.011 	 Acc: 0.997
[POSITIVE (1.00%)] Loss: 0.003 	 Acc: 1.000
[NEGATIVE (0.00%)] Loss: 1.973 	 Acc: 0.246
Fitness 0.950 	 LB Fitness: 0.896
 	 LB Fitness: 0.898

Runtime (prediction per trace):0.0000253525


RandomForestClassifier()

# 4. Train and Test

In [10]:
filename = "A_2017_im.csv"

for i in [2,4,6,8,10]:
    print("\n","m_AC=",i)
    
    # read the datafile, packageForFitness, minLengthRun, m_AC are needed to compute fitness and LBfitness
    x, y, packageForFitness, minLengthRun, m_AC  = cleanDataForRF("alignments-"+filename,i)
    
    # split the dataset in TRAIN and TEST sets
    X_train, X_test, y_train, y_test, packageForFitness_train, packageForFitness_test = train_test_split(x, y, packageForFitness, test_size=0.33, random_state=42)

    # ----------------------------------------------
    #                     TRAIN 
    # ----------------------------------------------
    # run the cross-validation
    bestmodel = runKFoldForRF(10, X_train, y_train, packageForFitness_train, minLengthRun, m_AC)
    
    # ----------------------------------------------
    #                     TEST 
    # ----------------------------------------------
    # use the same function as in the cross-validation but for the test set. 
    accAll, lossAll = [], []
    accNeg, lossNeg, percentageNeg   = [], [], []
    accPos, lossPos, percentagePos = [], [], []
    
    packageForFitness_test.reset_index(drop=True,inplace=True)
    print(packageForFitness_test.freqs.shape)
    #compute loss and accuracy on the test items 
    accLossPercentage(bestmodel, X_test, y_test, list(range(0,len(y_test))), accAll, lossAll)
        
    # compute loss and accuracy on the test items that are negatives
    accLossPercentage(bestmodel, X_test, y_test, list(range(0,len(y_test))), accNeg, lossNeg, percentageNeg,packageForFitness_test.freqs, 0)

    # compute loss and accuracy on the test items that are positives
    indices_of_pos = accLossPercentage(bestmodel, X_test, y_test, list(range(0,len(y_test))), accPos, lossPos, percentagePos,packageForFitness_test.freqs, 1) 

    # compute fitness and lower-bound
    realFitness = fitness(packageForFitness_test, minLengthRun)
    if len(indices_of_pos)>0:
        realLBFitness = LB_fitness(packageForFitness_test, minLengthRun, m_AC, indices_of_pos)
        
    # compute predicted LB fitness
    predictions = bestmodel.predict(X_test)
    indices_of_predicted_as_positives = [i for i in range(0, len(X_test)) if predictions[i][1]==1]
    if len(indices_of_predicted_as_positives)>0:
        predictedLBFitness = LB_fitness(packageForFitness_test, minLengthRun, m_AC, indices_of_predicted_as_positives)
        
    print("[TEST]\n[ALL] Loss:", "{:.3f}".format(mean(lossAll)), "\t Acc:", "{:.3f}".format(mean(accAll)))
    print("[POSITIVE ({:.2f}%)] Loss:".format(mean(percentagePos)), "{:.3f}".format(mean(lossPos)), "\t Acc:", "{:.3f}".format(mean(accPos)))
    print("[NEGATIVE ({:.2f}%)] Loss:".format(mean(percentageNeg)), "{:.3f}".format(mean(lossNeg)),"\t Acc:", "{:.3f}".format(mean(accNeg)))
    print("Fitness {:.3f}".format((realFitness)), "\t LB Fitness:", "{:.3f}\n".format((realLBFitness)),"\t Predicted LB Fitness:", "{:.3f}\n".format((predictedLBFitness)))
    
    fake_x, fake_y, fake_packageForFitness, fake_minLengthRun, fake_m_AC  = cleanDataForRF("alignments-mock-"+filename,i,vectorizer_already_trained=True)

    accFake, lossFake = [], []
    accLossPercentage(bestmodel, fake_x, fake_y, range(0,len(fake_y)), accFake, lossFake)
    print("[MOCK] Loss:", "{:.3f}".format(mean(lossFake)), "\t Acc:", "{:.3f}".format(mean(accFake)))
    
    print("-------------------------------------------")


 m_AC= 2
[CROSS-VALIDATION]
[ALL] Loss: 0.037 	 Acc: 0.991
[POSITIVE (0.07%)] Loss: 0.118 	 Acc: 0.958
[NEGATIVE (0.93%)] Loss: 0.030 	 Acc: 0.993
Fitness 0.873 	 LB Fitness: 0.073
 	 LB Fitness: 0.076

Runtime (prediction per trace):0.0000295276
(5257,)
[TEST]
[ALL] Loss: 0.033 	 Acc: 0.990
[POSITIVE (0.07%)] Loss: 0.098 	 Acc: 0.962
[NEGATIVE (0.93%)] Loss: 0.028 	 Acc: 0.993
Fitness 0.874 	 LB Fitness: 0.073
 	 Predicted LB Fitness: 0.077

[MOCK] Loss: 0.032 	 Acc: 0.998
-------------------------------------------

 m_AC= 4
[CROSS-VALIDATION]
[ALL] Loss: 0.042 	 Acc: 0.982
[POSITIVE (0.12%)] Loss: 0.124 	 Acc: 0.935
[NEGATIVE (0.88%)] Loss: 0.031 	 Acc: 0.988
Fitness 0.873 	 LB Fitness: 0.114
 	 LB Fitness: 0.116

Runtime (prediction per trace):0.0000281858
(5257,)
[TEST]
[ALL] Loss: 0.045 	 Acc: 0.982
[POSITIVE (0.12%)] Loss: 0.137 	 Acc: 0.941
[NEGATIVE (0.88%)] Loss: 0.032 	 Acc: 0.988
Fitness 0.874 	 LB Fitness: 0.115
 	 Predicted LB Fitness: 0.119

[MOCK] Loss: 0.080 	 Acc: 0.

# Bonus: Important Features

In [11]:
# ---- (3) prepare the sequences of activities 
data = pd.read_csv("alignments-"+filename,sep = ";", 
                   names = ["traces","tracesWithMoves","runs","runsWithMoves","costs","frequencies"])

traces_to_matrix = data.traces.str.split(":::",expand=True,)
number_of_traces, max_len = traces_to_matrix.shape
print("Number of traces:", number_of_traces, "\nMax len of traces:", max_len) 

# transform the matrix to a serie
traces_to_serie = pd.concat([traces_to_matrix[i] for i in range(0,max_len)], axis=0, 
                                      ignore_index=True, sort=False)
# from the serie, it's easy to get unique words
index_to_word = list(filter(None,(traces_to_serie).unique()))

Number of traces: 15930 
Max len of traces: 181


In [12]:
for feature in zip(index_to_word, np.sort(bestmodel.feature_importances_)):
    print(feature)

('A_Create Application', 0.0)
('A_Submitted', 0.0)
('W_Complete application', 0.0)
('A_Concept', 0.00010496334607664634)
('W_Handle leads', 0.0001918745521716391)
('A_Accepted', 0.0019968065082772613)
('W_Assess potential fraud', 0.0028161895108348625)
('O_Create Offer', 0.003149536613086957)
('O_Created', 0.004536081114568189)
('W_Shortened completion ', 0.004773947834213605)
('O_Sent (online only)', 0.005154637630305176)
('O_Sent (mail and online)', 0.005716025153827229)
('O_Cancelled', 0.005797008287282642)
('A_Cancelled', 0.008843930808015736)
('A_Denied', 0.010807772628315105)
('O_Refused', 0.018267269867160634)
('W_Call after offers', 0.02062744155335336)
('A_Complete', 0.022579932326591487)
('W_Validate application', 0.03419388784227591)
('A_Validating', 0.04055927800916162)
('O_Returned', 0.041344253073254)
('W_Call incomplete files', 0.04141863408779968)
('O_Accepted', 0.041864969568009835)
('A_Incomplete', 0.0494769255555316)
('A_Pending', 0.07705228005193387)
('W_Personal Lo